In [1]:
from flask import Flask, jsonify
import pandas as pd
import json
from collections import defaultdict
from datetime import datetime
import calendar
from aquasentinel import predict_wqi, get_wqc 
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from flask import request
import logging

In [69]:
data = pd.read_csv("../backend/dataset/indian_water_physicochemical_parameters.csv", encoding='unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
dates = data["date"].values


In [72]:
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,1024.0,"TAMBIRAPARANI AT SRIVAIKUNTAM, D/S OF SK ANAIC...",PUNJAB,23.6,4.5,5.7,6174.2,61.1,1.70,12240.5,47083.7,2024,2024-04-20
4140,2052.0,PERIYAR AT KALADY,MIZORAM,19.7,5.9,6.3,8788.6,40.2,11.40,8336.6,2856.9,2024,2024-04-21
4141,2709.0,"PATALGANGA RIVER AT VYAL PUMP HOUSE , KHALAPUR...",MAHARASHTRA,27.1,8.0,6.0,4720.8,48.8,0.60,25561.4,74069.8,2024,2024-04-22
4142,1246.0,RIVER GHAGGAR AT D/S OF SURAJPUR,GUJARAT,17.1,2.8,5.6,9496.6,68.5,5.20,10582.9,60918.8,2024,2024-04-23


In [73]:
import random
from datetime import datetime, timedelta

def generate_dates():
    start_date = datetime(2014, 12, 28)
    end_date = datetime.now()

    while start_date <= end_date:
        yield start_date.strftime("%Y-%m-%d")
        start_date += timedelta(days=1)

dates = list(generate_dates())

states = data["STATE"].dropna().unique()
locations = data["LOCATIONS"].dropna().unique()
state_station_code = data.groupby('STATE')['STATION CODE'].unique().apply(lambda x: random.choice(x)).to_dict()




max_temp = data["Temp"].max()
min_temp = data["Temp"].min()

max_do = data["D.O. (mg/l)"].max()
min_do = data["D.O. (mg/l)"].min()


max_ph = data["PH"].max()
min_ph = data["PH"].min()


max_cod = data["CONDUCTIVITY (Âµmhos/cm)"].max()
min_cod = data["CONDUCTIVITY (Âµmhos/cm)"].min()


max_bod = data["B.O.D. (mg/l)"].max()
min_bod = data["B.O.D. (mg/l)"].min()

max_nit = data["NITRATENAN N+ NITRITENANN (mg/l)"].max()
min_nit = data["NITRATENAN N+ NITRITENANN (mg/l)"].min()


max_fecal = data["FECAL COLIFORM (MPN/100ml)"].max()
min_fecal = data["FECAL COLIFORM (MPN/100ml)"].min()


max_total_col = data["TOTAL COLIFORM (MPN/100ml)Mean"].max()
min_total_col = data["TOTAL COLIFORM (MPN/100ml)Mean"].min()


for i in dates:
    random_state = random.choice(list(states))
    data.loc[len(data.index)] = [state_station_code[random_state],
                                 random.choice(list(locations)),
                                random_state,
                                round(random.uniform(min_temp, max_temp), 1),
                                round(random.uniform(min_do, max_do), 1),
                                round(random.uniform(min_ph, max_ph), 1),
                                round(random.uniform(min_cod, max_cod), 1),
                                round(random.uniform(min_bod, max_bod), 1),
                                round(random.uniform(min_nit, max_nit), 1),
                                round(random.uniform(min_fecal, max_fecal), 1),
                                round(random.uniform(min_total_col, max_total_col), 1),
                                i[0:4],
                                i]

In [74]:

data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7545,3305.0,R KUPPAM AT THALIPARAMBA,MADHYA PRADESH,30.1,3.7,7.7,10162.6,20.8,3.40,45407.2,75886.0,2024,2024-04-20
7546,1476.0,"MANDA RIVER (DUDHAI RIVER) AT JAMPA, EAST GARO...",GOA,27.7,4.1,8.0,17294.6,16.6,6.60,16011.3,59601.0,2024,2024-04-21
7547,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,26.5,6.7,8.3,3031.8,80.7,10.50,3861.8,21468.8,2024,2024-04-22
7548,1885.0,NAMBUL AT HEIRANGOITHONG,HARYANA,32.2,7.5,8.4,9170.6,50.1,6.50,5473.9,28491.1,2024,2024-04-23


In [75]:
data.dtypes

STATION CODE                        float64
LOCATIONS                            object
STATE                                object
Temp                                float64
D.O. (mg/l)                         float64
PH                                  float64
CONDUCTIVITY (Âµmhos/cm)            float64
B.O.D. (mg/l)                       float64
NITRATENAN N+ NITRITENANN (mg/l)    float64
FECAL COLIFORM (MPN/100ml)          float64
TOTAL COLIFORM (MPN/100ml)Mean      float64
year                                 object
date                                 object
dtype: object

In [76]:
data.to_csv("data_generation.csv", index=False)


In [77]:
data = pd.read_csv("../backend/dataset/data_generation.csv")

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7545,3305.0,R KUPPAM AT THALIPARAMBA,MADHYA PRADESH,30.1,3.7,7.7,10162.6,20.8,3.40,45407.2,75886.0,2024,2024-04-20
7546,1476.0,"MANDA RIVER (DUDHAI RIVER) AT JAMPA, EAST GARO...",GOA,27.7,4.1,8.0,17294.6,16.6,6.60,16011.3,59601.0,2024,2024-04-21
7547,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,26.5,6.7,8.3,3031.8,80.7,10.50,3861.8,21468.8,2024,2024-04-22
7548,1885.0,NAMBUL AT HEIRANGOITHONG,HARYANA,32.2,7.5,8.4,9170.6,50.1,6.50,5473.9,28491.1,2024,2024-04-23
